# Tarea II Inversiones

In [26]:
import numpy as np
import pandas as pd
from scipy.stats import norm
#Tarea PredictorData2021.xlsx comparación Pauta_Ayu1.xlsx, "Prediction"
df=pd.read_excel('PredictorData2021.xlsx',sheet_name='Monthly',header=0)
df.head()

C:\Users\AsusViHero\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,yyyymm,sp500,d12,e12,bm,tbl,aaa,baa,lty,ntis,rfree,infl,ltr,corpr,svar,crsp_spvw,vol
0,187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004967,NaN,NaN,NaN,NaN,NaN,NaN
2,187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004525,NaN,NaN,NaN,NaN,NaN,NaN
3,187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004252,NaN,NaN,NaN,NaN,NaN,NaN
4,187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004643,NaN,NaN,NaN,NaN,NaN,NaN


## a) 
Genere el logaritmo del exceso de retorno (equity premium) como:
$𝑟𝑡=𝑙𝑜𝑔(1+𝐶𝑅𝑆𝑃𝑡𝑆𝑃𝑣𝑤)−𝑙𝑜𝑔(1+𝑅𝑓𝑡−1)$
Donde 𝐶𝑅𝑆𝑃𝑡𝑆𝑃𝑣𝑤 es el equity premium incluyendo dividendos en 𝑡 y 𝑅𝑓𝑡−1 la tasa libre de riesgo (rfree) en 𝑡−1.

In [27]:
aux_crsp=df["crsp_spvw"][1:].reset_index().drop('index',axis=1)


aux_rfree=df["rfree"][:-1].reset_index().drop('index',axis=1)

aux_rt = np.log(1+aux_crsp["crsp_spvw"]) - np.log(1+aux_rfree["rfree"])
df["rt"] = pd.concat([pd.Series(np.nan),pd.Series(aux_rt)]).reset_index().drop('index',axis=1)
df["rt"]=df["rt"]

## b)
Basándose en Welch y Goyal (2008) construya y/o seleccione los siguientes predictores:

In [86]:
# i)
log_dp = np.log(df["d12"])-np.log(df["sp500"])
log_dp[1:]=log_dp[:-1]
log_dp[0]=np.nan
df["log_dp"] =log_dp

# ii)
rez_sp500 = df["sp500"][:-1].reset_index().drop('index',axis=1)
d12_aux = df["d12"][1:].reset_index().drop('index',axis=1)
log_dy = np.log(d12_aux["d12"])-np.log(rez_sp500["sp500"])

df["log_dy"]=pd.concat([pd.Series(np.nan),log_dy]).reset_index().drop('index',axis=1)
# iii)
log_ep = np.log(df["e12"])-np.log(df["sp500"])
log_ep[1:]=log_ep[:-1]
log_ep[0]=np.nan
df["log_ep"]= log_ep
# iv)
log_de = np.log(df["d12"])-np.log(df["e12"])
log_de[1:]=log_de[:-1]
log_de[0]=np.nan
df["log_de"]= log_de
# xi)
tms = df["lty"] - df["tbl"]
tms[1:]=tms[:-1]
tms[0]=np.nan
df["tms"]=tms*100
# xii)
dfy = df["baa"]-df["aaa"]
dfy[1:]=dfy[:-1]
dfy[0]=np.nan
df["dfy"]=dfy*100
# xii)
dfr=df["corpr"]-df["ltr"]
dfr[1:]=dfr[:-1]
dfr[0]=np.nan
df["dfr"]=dfr*100
# xii)
infl_lag=pd.DataFrame.copy(df["infl"])
infl_lag[1:]=infl_lag[:-1]
infl_lag[0]=np.nan
df["infl_lag"]=infl_lag*100

df.loc[1809:,['yyyymm',"log_dp","log_dy","log_ep","log_de",'svar',"bm","ntis","tbl","lty","ltr","tms","dfy","dfr","infl_lag"]]

,yyyymm,log_dp,log_dy,log_ep,log_de,svar,bm,ntis,tbl,lty,ltr,tms,dfy,dfr,infl_lag
1809,202110,-4.286281,-4.279874,-3.201224,-1.085057,0.001151,0.182389,0.013368,0.0005,0.0158,0.0051,1.33,0.70,0.56,0.271597
1810,202111,-4.346732,-4.340365,-3.226260,-1.120472,0.001327,0.189455,0.015640,0.0005,0.0156,0.0210,1.53,0.67,1.08,0.830812
1811,202112,-4.331997,-4.325670,-3.177747,-1.154249,0.002518,0.179786,0.012970,0.0006,0.0146,-0.0110,1.51,0.66,-1.16,0.491343


## C)
Creación de Indicadores Técnicos

# Media Móvil


In [4]:
ma=pd.DataFrame(np.zeros(shape=(len(df),5)))
ma.rename(columns={0:"MA_1",1:"MA_2",2:"MA_3",3:"MA_9",4:"MA_12"},inplace=True)


In [5]:
ma["MA_1"]=pd.DataFrame.copy(df["sp500"])
obs=len(df)
MA_2 = np.zeros(shape=(obs,1))
for i in range(obs):
    if i==0:
        MA_2[i]=np.nan
    else:
        MA_2[i]=(1/2)*(df["sp500"][i]+df["sp500"][i-1])
ma["MA_2"]=MA_2


MA_3 = np.zeros(shape=(obs,1))
for i in range(obs):
    if i < 2:
        MA_3[i] = np.nan
    else:
        MA_3[i]=(1/3)*(np.sum(df["sp500"][i-2:i+1]))
ma["MA_3"] = MA_3

MA_9 = np.zeros(shape=(obs,1))
for i in range(obs):
    if i < 8:
        MA_9[i] = np.nan
    else:
        MA_9[i]=(1/9)*(np.sum(df["sp500"][i-8:i+1]))
ma["MA_9"] = MA_9

MA_12 = np.zeros(shape=(obs,1))
for i in range(obs):
    if i < 11:
        MA_12[i] = np.nan
    else:
        MA_12[i]=(1/12)*(np.sum(df["sp500"][i-11:i+1]))
ma["MA_12"] = MA_12
ma["MA_12"]

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
           ...     
1807    3925.805000
1808    4004.516667
1809    4115.801667
1810    4194.582500
1811    4278.758333
Name: MA_12, Length: 1812, dtype: float64

In [6]:
ma["S_1_9"]=np.array(ma["MA_1"]>=ma["MA_9"]).astype("int")

ma["S_1_12"]=np.array(ma["MA_1"]>=ma["MA_12"]).astype("int")

ma["S_2_9"]=np.array(ma["MA_2"]>=ma["MA_9"]).astype("int")

ma["S_2_12"]=np.array(ma["MA_2"]>=ma["MA_12"]).astype("int")

ma["S_3_9"]=np.array(ma["MA_3"]>=ma["MA_9"]).astype("int")

ma["S_3_12"]=np.array(ma["MA_3"]>=ma["MA_12"]).astype("int")
for i in ["S_1_9","S_1_12","S_2_9","S_2_12","S_3_9","S_3_12"]:
    df[i]=ma[i]


## Momentum

In [7]:
MOM_9=np.empty(shape=(obs,1))
for i in range(obs):
    if i < 9:
        MOM_9[i]=np.nan
    else:
        MOM_9[i]=int(df["sp500"][i-1]>=df["sp500"][i-9])
        
assert np.sum(np.isnan(MOM_9)) == 9

MOM_12=np.empty(shape=(obs,1))
for i in range(obs):
    if i < 12:
        MOM_12[i]=np.nan
    else:
        MOM_12[i]=int(df["sp500"][i-1]>=df["sp500"][i-12])
        
assert np.sum(np.isnan(MOM_12)) == 12
df["mom_9"]=MOM_9
df["mom_12"]=MOM_12

## Volume

In [8]:
#obv=pd.DataFrame(np.zeros(shape=(len(df),1)))
D_k=np.zeros(shape=(obs,1))
for i in range(obs):
    if i==0:
        D_k[i]=np.nan
    else:
        D_k[i]=int(df["sp500"][i]>=df["sp500"][i-1])
D_k[D_k==0]=-1

aux_obv= np.zeros(shape=(obs,1))
for i in range(obs):
    if i==0:
        aux_obv[i]=np.nan
    else:
        aux_obv[i]=df["vol"][i]*D_k[i]
obv_t=np.zeros(shape=(obs,1))

for i in range(obs):
    obv_t[i]=np.nansum(aux_obv[0:i+1])
ma_obv=pd.DataFrame(np.zeros(shape=(obs,5)))
ma_obv.rename(columns={0:"MA_obv_1",1:"MA_obv_2",2:"MA_obv_3",3:"MA_obv_9",4:"MA_obv_12"},inplace=True)
ma_obv["MA_obv_1"]=obv_t


In [9]:
ma_obv_2=np.zeros(shape=(obs,1))
for i in range(obs):
    if i==0:
        ma_obv_2[i]=np.nan
    else:
        ma_obv_2[i]=(1/2)*(obv_t[i]+obv_t[i-1])
        
ma_obv["MA_obv_2"]=ma_obv_2

ma_obv_3=np.zeros(shape=(obs,1))
for i in range(obs):
    if i<2:
        ma_obv_3[i]=np.nan
    else:
        ma_obv_3[i]=(1/3)*(np.nansum(obv_t[i-2:i+1]))
        
ma_obv["MA_obv_3"]=ma_obv_3


ma_obv_9=np.zeros(shape=(obs,1))
for i in range(obs):
    if i<8:
        ma_obv_9[i]=np.nan
    else:
        ma_obv_9[i]=(1/9)*(np.nansum(obv_t[i-8:i+1]))
        
ma_obv["MA_obv_9"]=ma_obv_9


ma_obv_12=np.zeros(shape=(obs,1))
for i in range(obs):
    if i<11:
        ma_obv_12[i]=np.nan
    else:
        ma_obv_12[i]=(1/12)*(np.nansum(obv_t[i-11:i+1]))
        
ma_obv["MA_obv_12"]=ma_obv_12

In [10]:
ma_obv["S_obv_1_9"]=np.array(ma_obv["MA_obv_1"]>=ma_obv["MA_obv_9"]).astype("int")

ma_obv["S_obv_1_12"]=np.array(ma_obv["MA_obv_1"]>=ma_obv["MA_obv_12"]).astype("int")

ma_obv["S_obv_2_9"]=np.array(ma_obv["MA_obv_2"]>=ma_obv["MA_obv_9"]).astype("int")

ma_obv["S_obv_2_12"]=np.array(ma_obv["MA_obv_2"]>=ma_obv["MA_obv_12"]).astype("int")

ma_obv["S_obv_3_9"]=np.array(ma_obv["MA_obv_3"]>=ma_obv["MA_obv_9"]).astype("int")

ma_obv["S_obv_3_12"]=np.array(ma_obv["MA_obv_3"]>=ma_obv["MA_obv_12"]).astype("int")
for i in ["S_obv_1_9","S_obv_1_12","S_obv_2_9","S_obv_2_12","S_obv_3_9","S_obv_3_12"]:
    df[i]=ma_obv[i]


In [11]:
hst_avg=np.zeros(shape=(obs,1));
for i in range(obs):
    if i==0:
        hst_avg[i]=np.nan
    else:
        hst_avg[i]=df.loc[1:i,"rt"].mean()
hst_avg[1:]=hst_avg[:-1]
hst_avg[0]=np.nan
df["hst_avg"]=hst_avg
df[["rt","hst_avg"]]

,rt,hst_avg
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1807,0.030141,0.005453
1808,-0.047171,0.005474
1809,0.068135,0.005428
1810,-0.007282,0.005483


## D) Estadística Descriptiva

In [12]:
#Selección de datos relevantes
#tarea 196812 ,comparación 192611
datos_relevantes=196812
df=df.loc[df['yyyymm']>=datos_relevantes,:]
# Ajustar los predictores en t con sus rezagos para realizar posteriormente las estimaciones
pred_ajustar=["svar","bm","ntis","tbl","lty","ltr"]
for i in pred_ajustar:
    aux_prd=pd.DataFrame.copy(df[i])
    aux_prd[1:]=df[i][:-1]
    aux_prd[0]=np.nan
    df[i]=aux_prd
df[["rt","log_dp","log_dy","log_ep","log_de",'svar',"bm","ntis","tbl","lty","ltr","tms","dfy","dfr","infl_lag","S_1_9","S_1_12","S_2_9","S_2_12","S_3_9","S_3_12","mom_9","mom_12","S_obv_1_9","S_obv_1_12","S_obv_2_9","S_obv_2_12","S_obv_3_9","S_obv_3_12"]].apply(['mean','std','skew','kurtosis','autocorr']).T.round(2)

,mean,std,skew,kurtosis,autocorr
rt,0.00,0.04,-0.69,2.39,0.04
log_dp,-3.65,0.42,0.09,-1.01,0.99
log_dy,-3.65,0.42,0.09,-1.00,0.99
log_ep,-2.87,0.46,-0.51,1.96,0.99
log_de,-0.78,0.32,3.07,16.57,0.99
svar,0.00,0.01,9.75,114.54,0.39
bm,0.47,0.28,0.89,-0.39,0.99
ntis,0.01,0.02,-0.41,-0.29,0.98
tbl,0.05,0.03,0.60,0.32,0.99
lty,0.06,0.03,0.36,-0.17,0.99


# Parte 2

In [13]:
def MCO(var_depe,var_indepe,test_t):
    

    if test_t==True:
        x_tx_inv=np.linalg.inv(np.matmul(var_indepe.T,var_indepe))
        x_ty=np.matmul(var_indepe.T,var_depe)
        coef_mco=np.matmul(x_tx_inv,x_ty)
        resid=var_depe-coef_mco*var_indepe
        resid_2=np.matmul(resid.T,resid)
        sigma_2=resid_2/(len(var_depe)-len(coef_mco))
        var_beta= sigma_2*x_tx_inv
        test_t_cte=  (coef_mco[0])/(np.sqrt(var_beta[0]))
        p_value = 1-norm.cdf(test_t_cte)
        return coef_mco.T,test_t_cte,p_value
    else:
        cte=np.ones(shape=(len(var_indepe),1))
        x=np.column_stack((cte,var_indepe))
        x_tx_inv=np.linalg.inv(np.matmul(x.T,x))
        x_ty=np.matmul(x.T,var_depe)
        coef_mco=np.matmul(x_tx_inv,x_ty)
        return coef_mco.T


Período in sample es 1968:12-1989:12

In [14]:
df.reset_index(drop=True,inplace=True)
## Tarea 198912, comparación 195701
fecha_quiebre=198912
pos_quiebre=df.loc[df["yyyymm"]==fecha_quiebre].index[0]
n_regresiones= len(df.loc[df["yyyymm"]>=fecha_quiebre])
predictores=["log_dp","log_dy","log_ep","log_de",'svar',"bm","ntis","tbl","lty","ltr","tms","dfy","dfr","infl_lag","S_1_9","S_1_12","S_2_9","S_2_12","S_3_9","S_3_12","mom_9","mom_12","S_obv_1_9","S_obv_1_12","S_obv_2_9","S_obv_2_12","S_obv_3_9","S_obv_3_12"]
R_2_os_dict=dict.fromkeys(predictores,0)
for x in predictores:
    errores_modelo=np.zeros(shape=(n_regresiones,1))
    errores_benchmark=np.zeros(shape=(n_regresiones,1))
    d_test=np.zeros(shape=(n_regresiones,1))
    for i in range(n_regresiones):
        coef_mco= MCO(df["rt"][1:pos_quiebre+i],df[x][1:pos_quiebre+i],False)
        r_predicho=coef_mco[0]+coef_mco[1]*df[x][pos_quiebre+i]
        error_modelo=df["rt"][pos_quiebre+i]-r_predicho
        errores_modelo[i]=error_modelo**2
        error_benchmark=df["rt"][pos_quiebre+i]-df["hst_avg"][pos_quiebre+i]
        errores_benchmark[i]=error_benchmark**2
        d_test[i]=errores_modelo[i]-(errores_benchmark[i]-(df["hst_avg"][pos_quiebre+i]-r_predicho)**2)
    [coef_mco,est_msfe_adjusted,p_value]=MCO(d_test,np.ones(shape=(len(d_test),1)),True)
    R_2_os=1-(errores_modelo.mean()/errores_benchmark.mean())
    R_2_os_dict[x]=[R_2_os,est_msfe_adjusted[0],p_value[0]]

In [15]:
R_2_os_dict

{'log_dp': [-0.022004099577566105, 3.546332042484551, 0.0001953168324656529],
 'log_dy': [nan, nan, nan],
 'log_ep': [-0.013644999539330627, 1.7412708990452277, 0.04081805234754332],
 'log_de': [-0.018585056280441936, 1.5960147009754797, 0.05524275558816272],
 'svar': [-0.030908033785697953, 1.166759850539469, 0.12165368257771825],
 'bm': [-0.009790914646691462, 2.241507628467695, 0.01249660597493818],
 'ntis': [-0.023942955254963927, 3.841244963618677, 6.120593871905555e-05],
 'tbl': [-0.0013935531416036095, 0.8364479226369397, 0.201451481155695],
 'lty': [-0.002327523057569225, 0.8539913637697129, 0.1965548872750058],
 'ltr': [-0.011348131928851313, 1.903799394342668, 0.028468157726606047],
 'tms': [-0.016111267234194537, 2.7346734095483916, 0.003122109776975024],
 'dfy': [-0.032341706211328036, 4.175855001548932, 1.4843444108070258e-05],
 'dfr': [-0.013404782066056997, 1.1832979441849623, 0.11834554364260241],
 'infl_lag': [-0.01577714868284308, 2.3355313054834625, 0.009757841234020

In [16]:
df["log_dy"]

0     -3.518114
1     -3.506655
2     -3.454891
3     -3.485535
4     -3.503573
         ...   
632   -4.335019
633   -4.279874
634   -4.340365
635   -4.325670
636         NaN
Name: log_dy, Length: 637, dtype: float64

In [17]:
#Por si es necesario solo considerar la media histórica de los datos in sample
df=pd.read_excel('Prueba_preg2.xlsx',header=0)
aux_crsp=df["crsp_spvw"][1:].reset_index().drop('index',axis=1)


aux_rfree=df["rfree"][:-1].reset_index().drop('index',axis=1)

aux_rt = np.log(1+aux_crsp["crsp_spvw"]) - np.log(1+aux_rfree["rfree"])
df["rt"] = pd.concat([pd.Series(np.nan),pd.Series(aux_rt)]).reset_index().drop('index',axis=1)
log_dp = np.log(df["d12"])-np.log(df["sp500"])
log_dp[1:]=log_dp[:-1]
log_dp[0]=np.nan
df["log_dp"] =log_dp

df=df.loc[(df['yyyymm']>192611) & (df['yyyymm']<=201012),["yyyymm","rt","log_dp"]]
df.reset_index(drop=True,inplace=True)

obs=len(df)
# Media Histórica
## Tarea 198912, comparación 195701
fecha_quiebre=195701
n_regresiones= len(df.loc[df["yyyymm"]>=fecha_quiebre])

pos_quiebre=df.loc[df["yyyymm"]==fecha_quiebre].index[0]
hst_avg=np.zeros(shape=(obs,1));
for i in range(obs):
    if i<pos_quiebre-1:
        hst_avg[i]=np.nan
    else:
        hst_avg[i]=df.loc[0:i,"rt"].mean()
hst_avg[1:]=hst_avg[:-1]
hst_avg[0]=np.nan
df["hst_avg"]=hst_avg



predictores=["log_dp"]
R_2_os_dict=dict.fromkeys(predictores,0)
for x in predictores:
    errores_modelo=np.zeros(shape=(n_regresiones,1))
    errores_benchmark=np.zeros(shape=(n_regresiones,1))
    d_test=np.zeros(shape=(n_regresiones,1))
    for i in range(n_regresiones):
        coef_mco= MCO(df["rt"][1:pos_quiebre+i],df[x][1:pos_quiebre+i],False)
        r_predicho=coef_mco[0]+coef_mco[1]*df[x][pos_quiebre+i]
        error_modelo=df["rt"][pos_quiebre+i]-r_predicho
        errores_modelo[i]=error_modelo**2
        error_benchmark=df["rt"][pos_quiebre+i]-df["hst_avg"][pos_quiebre+i]
        errores_benchmark[i]=error_benchmark**2
        d_test[i]=errores_modelo[i]-(errores_benchmark[i]-(df["hst_avg"][pos_quiebre+i]-r_predicho)**2)
    [coef_mco,est_msfe_adjusted,p_value]=MCO(d_test,np.ones(shape=(len(d_test),1)),True)
    R_2_os=1-(errores_modelo.mean()/errores_benchmark.mean())
    R_2_os_dict[x]=[R_2_os,est_msfe_adjusted[0],p_value[0]]
R_2_os_dict


{'log_dp': [-0.00048291040660775053, 1.4238146050697267, 0.0772500733631103]}

# Comparación con ayudantía

In [18]:
import numpy as np
import pandas as pd
from scipy.stats import norm
#Tarea PredictorData2021.xlsx comparación Pauta_Ayu1.xlsx, "Prediction"
df=pd.read_excel('Pauta_Ayu1.xlsx',sheet_name='Prediction',header=0,skiprows=1)
df.head()

,yyyymm,crsp_spvw,sp500,d12,rfree,Unnamed: 5,y,ha,x1,x1_lag,...,R2,f_hat,ones,cw,p_value,Unnamed: 22,yyyymm.1,date,diff,cumulative
0,192611,0.027014,13.27,0.6825,0.002792,NaN,NaN,NaN,NaN,NaN,...,-0.000483,0.000512,1.0,1.306716,0.095655,NaN,195701.0,1957-01,0.000486,0.000486
1,192612,0.026079,13.49,0.6900,0.002558,NaN,0.022957,NaN,-2.973012,NaN,...,NaN,0.000319,1.0,NaN,NaN,NaN,195702.0,1957-02,0.000297,0.000783
2,192701,-0.002638,13.21,0.6967,0.002692,NaN,-0.005197,NaN,-2.942374,-2.973012,...,NaN,-0.000123,1.0,NaN,NaN,NaN,195703.0,1957-03,-0.000141,0.000642
3,192702,0.045777,13.84,0.7033,0.002742,NaN,0.042072,NaN,-2.979535,-2.942374,...,NaN,-0.000328,1.0,NaN,NaN,NaN,195704.0,1957-04,-0.000348,0.000293
4,192703,0.007338,13.93,0.7100,0.002667,NaN,0.004573,NaN,-2.976535,-2.979535,...,NaN,-0.000284,1.0,NaN,NaN,NaN,195705.0,1957-05,-0.000308,-0.000014


In [19]:
aux_crsp=df["crsp_spvw"][1:].reset_index().drop('index',axis=1)


aux_rfree=df["rfree"][:-1].reset_index().drop('index',axis=1)

aux_rt = np.log(1+aux_crsp["crsp_spvw"]) - np.log(1+aux_rfree["rfree"])
df["rt"] = pd.concat([pd.Series(np.nan),pd.Series(aux_rt)]).reset_index().drop('index',axis=1)
df["rt"]=df["rt"]
log_dp = np.log(df["d12"])-np.log(df["sp500"])
log_dp[1:]=log_dp[:-1]
log_dp[0]=np.nan
df["log_dp"] =log_dp
df.loc[:,["y","x1_lag","rt","log_dp"]]

,y,x1_lag,rt,log_dp
0,NaN,NaN,NaN,NaN
1,0.022957,NaN,0.022957,-2.967499
2,-0.005197,-2.973012,-0.005197,-2.973012
3,0.042072,-2.942374,0.042072,-2.942374
4,0.004573,-2.979535,0.004573,-2.979535
...,...,...,...,...
1005,-0.046632,-3.906823,-0.046632,-3.906823
1006,0.086396,-3.853501,0.086396,-3.853501
1007,0.037880,-3.932741,0.037880,-3.932741
1008,-0.000163,-3.963373,-0.000163,-3.963373


In [20]:
obs=len(df)
hst_avg=np.zeros(shape=(obs,1));
for i in range(obs):
    if i==0:
        hst_avg[i]=np.nan
    else:
        hst_avg[i]=df.loc[1:i,"rt"].mean()
hst_avg[1:]=hst_avg[:-1]
hst_avg[0]=np.nan
df["hst_avg"]=hst_avg
df.loc[df['yyyymm']>=195701,["ha","hst_avg"]]

,ha,hst_avg
362,0.006627,0.006627
363,0.006488,0.006488
364,0.006394,0.006394
365,0.006434,0.006434
366,0.006533,0.006533
...,...,...
1005,0.004554,0.004554
1006,0.004503,0.004503
1007,0.004584,0.004584
1008,0.004617,0.004617


In [21]:
df.reset_index(drop=True,inplace=True)
## Tarea 198912, comparación 195701
fecha_quiebre=195701
pos_quiebre=df.loc[df["yyyymm"]==fecha_quiebre].index[0]
n_regresiones= len(df.loc[df["yyyymm"]>=fecha_quiebre])
predictores=["log_dp"]
R_2_os_dict=dict.fromkeys(predictores,0)
for x in predictores:
    errores_modelo=np.zeros(shape=(n_regresiones,1))
    errores_benchmark=np.zeros(shape=(n_regresiones,1))
    d_test=np.zeros(shape=(n_regresiones,1))
    for i in range(n_regresiones):
        coef_mco= MCO(df["rt"][2:pos_quiebre+i],df[x][2:pos_quiebre+i],False)
        r_predicho=coef_mco[0]+coef_mco[1]*df[x][pos_quiebre+i]
        error_modelo=df["rt"][pos_quiebre+i]-r_predicho
        errores_modelo[i]=error_modelo**2
        error_benchmark=df["rt"][pos_quiebre+i]-df["hst_avg"][pos_quiebre+i]
        errores_benchmark[i]=error_benchmark**2
        d_test[i]=errores_modelo[i]-(errores_benchmark[i]-(df["hst_avg"][pos_quiebre+i]-r_predicho)**2)
    [coef_mco,est_msfe_adjusted,p_value]=MCO(d_test,np.ones(shape=(len(d_test),1)),True)
    R_2_os=1-(errores_modelo.mean()/errores_benchmark.mean())
    R_2_os_dict[x]=[R_2_os,est_msfe_adjusted[0],p_value[0]]

In [22]:
R_2_os_dict[x]

[-0.00048291040660775053, 1.4238146050697267, 0.0772500733631103]

In [23]:
coef_mco

array([[2.11304949e-05]])

In [24]:
coef_mco= MCO(df["rt"][2:pos_quiebre],df[x][2:pos_quiebre],False)
r_predicho=coef_mco[0]+coef_mco[1]*df[x][pos_quiebre]
r_predicho

0.001546281484493324